# 1. Construir un sistema RAG que permita buscar información en una base de datos real de noticias, recuperando los artículos más relevantes y generando respuestas usando GPT-4.

* Usa LangChain y FAISS para la recuperación.
* Procesa un dataset real de noticias.
* Genera respuestas basadas en noticias relevantes.
* Evalúa la precisión con métricas como Recall@K y ROUGE.

In [1]:
# 1. Instalación de dependencias
%pip install chromadb langchain langchain_community openai pandas tiktoken

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# 2 - Descarga y Preprocesamiento del Dataset

Usaremos el dataset "News Category Dataset" de Kaggle, que contiene noticias reales.

Descarga el dataset desde Kaggle:

https://www.kaggle.com/datasets/rmisra/news-category-dataset?resource=download

descarga el archivo: News_Category_Dataset_v3.json.zip

# Cargar y Preprocesar el Dataset

El dataset de noticias viene en formato JSON, por lo que lo cargaremos y seleccionaremos las columnas necesarias.

In [2]:
import pandas as pd
import json

# Cargar el dataset de noticias
file_path = "News_Category_Dataset_v3.json"  # Asegúrate de cambiar esto si el nombre del archivo es diferente

# Leer el archivo JSON línea por línea
data = []
with open(file_path, "r") as file:
    for line in file:
        data.append(json.loads(line))

# Convertir a un DataFrame de pandas
df = pd.DataFrame(data)

# Seleccionar columnas relevantes
df = df[['headline', 'short_description', 'category']]
df = df.dropna()  # Eliminar filas con valores nulos

# Crear una columna combinada con el título y la descripción para indexación
df["content"] = df["headline"] + ". " + df["short_description"]

print(df.head())

# volverlo más pequeño para poderlo procesar en tiempos razonables de clase y en una laptop
df = df[:10000]


                                            headline  \
0  Over 4 Million Americans Roll Up Sleeves For O...   
1  American Airlines Flyer Charged, Banned For Li...   
2  23 Of The Funniest Tweets About Cats And Dogs ...   
3  The Funniest Tweets From Parents This Week (Se...   
4  Woman Who Called Cops On Black Bird-Watcher Lo...   

                                   short_description   category  \
0  Health experts said it is too early to predict...  U.S. NEWS   
1  He was subdued by passengers and crew when he ...  U.S. NEWS   
2  "Until you have a dog you don't understand wha...     COMEDY   
3  "Accidentally put grown-up toothpaste on my to...  PARENTING   
4  Amy Cooper accused investment firm Franklin Te...  U.S. NEWS   

                                             content  
0  Over 4 Million Americans Roll Up Sleeves For O...  
1  American Airlines Flyer Charged, Banned For Li...  
2  23 Of The Funniest Tweets About Cats And Dogs ...  
3  The Funniest Tweets From Parents This

In [3]:
print(df.describe)

<bound method NDFrame.describe of                                                headline  \
0     Over 4 Million Americans Roll Up Sleeves For O...   
1     American Airlines Flyer Charged, Banned For Li...   
2     23 Of The Funniest Tweets About Cats And Dogs ...   
3     The Funniest Tweets From Parents This Week (Se...   
4     Woman Who Called Cops On Black Bird-Watcher Lo...   
...                                                 ...   
9995  CDC Director Requests Salary Cut After Scrutin...   
9996  Super PAC Screw-Up: Ad Favorably Compares GOP ...   
9997  'Ant-Man And The Wasp' Trailer Brings The Fun ...   
9998  Here's What You Missed While The Internet Was ...   
9999  Facebook Will Let Users See Which Sites Are Tr...   

                                      short_description       category  \
0     Health experts said it is too early to predict...      U.S. NEWS   
1     He was subdued by passengers and crew when he ...      U.S. NEWS   
2     "Until you have a dog you don

# 3. Crear la Base de Datos Vectorial en ChromaDB

Usamos ChromaDB para almacenar y recuperar embeddings de noticias.

In [7]:
import chromadb
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv('api_keys.env')
# Configurar la API Key (puedes establecerla como variable de entorno)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Inicializar ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_news_db")

# Crear una colección en ChromaDB
collection = chroma_client.get_or_create_collection(name="news_articles")

# Generar embeddings para cada noticia y almacenarlas en la base de datos
embedding_model = OpenAIEmbeddings()
i=0
# en una laptop 16 GB RAM Intel i5, 35seg por cada 100 registros, 
# cargar 10.000 registros, toma aprox = 1 hora
for idx, row in df.iterrows():
    if (i%100==0):
        print(i)
    i=i+1
    collection.add(
        ids=[str(idx)],
        documents=[row["content"]],
        metadatas=[{"category": row["category"], "headline": row["headline"]}]
    )

print("Base de datos de noticias creada con éxito.")


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
Base de datos de noticias creada con éxito.


Inicializamos ChromaDB en modo persistente.
Generamos embeddings con OpenAI y almacenamos los textos en ChromaDB.
Guardamos metadatos como categoría y título para mejorar la interpretación de los resultados.

# 4. Búsqueda Semántica con Recuperación de Noticias

Ahora creamos una función para buscar las noticias más relevantes usando búsqueda vectorial en ChromaDB.

In [8]:
def search_news(query, top_k=3):
    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )
    return results["documents"][0], results["metadatas"][0]

# Prueba de búsqueda
query = "News about Trump, white house"
retrieved_docs, metadata = search_news(query)

print("\nNoticias recuperadas:")
for i in range(len(retrieved_docs)):
    print(f"\n{metadata[i]['headline']}")
    print(f"\n{retrieved_docs[i]}")



Noticias recuperadas:

White House Press Secretary Goofs Up, Broadcasts Trump's Banking Details

White House Press Secretary Goofs Up, Broadcasts Trump's Banking Details. Trump's donation to the Department of Health and Human Services shown off by Kayleigh McEnany included a few too many details.

No White House Progress On Day 3 Of Government Shutdown

No White House Progress On Day 3 Of Government Shutdown. “Nothing new. Nothing new on the shutdown. Nothing new. Except we need border security,” Trump told reporters at the White House

Trump's Silent Public Outing Belies White House In Tumult

Trump's Silent Public Outing Belies White House In Tumult. The president's appearance at Arlington National Cemetery was his first public outing for official business in more than a week.


search_news() consulta ChromaDB para recuperar los artículos más relevantes.
Ejemplo de consulta: Buscamos noticias sobre "elecciones en EE.UU." y mostramos los títulos y descripciones.

# 5. Generación de Respuestas con OpenAI GPT-4

Integramos ahora la recuperación con GPT-4 para responder preguntas basadas en las noticias.

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma

# Configurar el modelo de lenguaje
llm = ChatOpenAI(model_name="gpt-4")

# Crear la función de RAG
def rag_query(user_query, top_k=3):
    retrieved_docs, metadata = search_news(user_query, top_k)
    
    # Formatear el contexto para el modelo GPT-4
    context = "\n".join(retrieved_docs)
    
    # Crear el prompt para el modelo
    prompt = f"""
    Basado en las siguientes noticias, responde la siguiente pregunta de manera clara y concisa:

    Noticias:
    {context}

    Pregunta: {user_query}
    """

    # Generar la respuesta
    response = llm.invoke(prompt)
    
    return response

# Prueba de generación
user_query = "¿Cuáles son las últimas noticias sobre el cambio climático?"
response = rag_query(user_query)

print("\nRespuesta generada por RAG:")
print(response)



Respuesta generada por RAG:
content='Las últimas noticias sobre el cambio climático se relacionan con los esfuerzos de Biden para abordar este problema a nivel global. Específicamente, se está centrando en Brasil, ya que es un país clave para sus ambiciones climáticas. Sin embargo, está encontrando problemas para lograr que el presidente de extrema derecha de Brasil, Bolsonaro, se preocupe por la deforestación.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 180, 'total_tokens': 271, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-b748f4cd-9bd7-43f5-8dba-b1b2d0fb43d8-0'


Usamos GPT-4 para generar respuestas basadas en las noticias recuperadas.
rag_query() recupera noticias, formatea un prompt y consulta a GPT-4.
Prueba con consulta real: "¿Cuáles son las últimas noticias sobre el cambio climático?"

# 6. Evaluación del Sistema RAG

Evaluamos la recuperación y generación con Recall@K y ROUGE.

## 6.1 Evaluación de la Recuperación con Recall@K

In [11]:
def evaluate_recall(queries, ground_truths, k=3):
    retrieved_texts = [search_news(q, k)[0] for q in queries]
    recall_k = sum(any(gt in retrieved for retrieved in retrieved_texts) for gt in ground_truths) / len(queries)
    
    print(f"\nRecall {k}: {recall_k:.2f}")

# Definir consultas y respuestas esperadas
test_queries = ["Noticias sobre economía", "Últimos eventos deportivos"]
expected_answers = ["Economía en crecimiento", "Partido de fútbol"]

evaluate_recall(test_queries, expected_answers, k=3)



Recall 3: 0.00


In [15]:
!pip install evaluate
!pip install bert_score


[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 61.1/61.1 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 8.1/8.1 MB 21.5 MB/s eta 0:00:00
     ------------------------------------- 222.0/222.0 kB 13.2 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 28.1 MB/s eta 0:00:00
     ---------------------------------------- 72.0/72.0 kB 3.9 MB/s eta 0:00:00
     -------------------------------------- 111.1/111.1 kB 6.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Recall@K mide la capacidad del sistema de recuperar información correcta.
Si el recall es alto (cercano a 1), la recuperación es precisa.

## 6.2 Evaluación de la Generación con ROUGE

In [23]:
import evaluate

# 1. Load BERTScore metric
bertscore = evaluate.load("bertscore")


# Comparar respuestas generadas vs. esperadas
reference = ["El cambio climático está afectando las temperaturas globales."]
generated = [rag_query("¿Cómo afecta el cambio climático al planeta?").content]

results = bertscore.compute(
    predictions=generated,
    references=reference,
    lang="es"  # "en" for English, "es" for Spanish
)


# 4. See the results
print(f"Precision: {results['precision']}")
print(f"Recall: {results['recall']}")
print(f"F1: {results['f1']}")


c:\Users\jdmartinev\OneDrive - Universidad EAFIT\Cursos\Inteligencia Artificial - IM\Lecture13\notebooks\ragenv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jdmartinev\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled f

Precision: [0.6775435209274292]
Recall: [0.8214945793151855]
F1: [0.7426072955131531]


In [22]:
reference

'El cambio climático está afectando las temperaturas globales.'

ROUGE mide la similitud entre la respuesta generada y la respuesta esperada.
ROUGE alto indica que el modelo genera respuestas precisas.

# Conclusiones

* Creamos un sistema RAG real con ChromaDB y OpenAI para búsqueda de noticias.
* Probamos consultas reales y generamos respuestas con GPT-4.
* Evaluamos la recuperación con Recall@K y la generación con ROUGE.

RETO: (1) actualización automática de noticias - (2) visualización de embeddings